### Project Setup

First run the following steps only if you are running Datalab from your local desktop or laptop (not running Datalab from a GCE VM):

1. Make sure you have a GCP project which is enabled for Machine Learning API and Dataflow API.
2. Run "%datalab project set --project [project-id]" to set the default project in Datalab.

If you run Datalab from a GCE VM, then make sure the project of the GCE VM is enabled for Machine Learning API and Dataflow API.

In [1]:
%load_ext google.datalab.kernel

In [2]:
%datalab project set --project ajhamilton-scratch

In [3]:
bucket = 'gs://' + datalab_project_id() + '-coast'

In [4]:
!gsutil mb $bucket

Creating gs://ajhamilton-scratch-coast/...


### Data Preparation

All data is under gs://cloud-datalab/sampledata/coast. See https://storage.googleapis.com/tamucc_coastline/GooglePermissionForImages_20170119.pdf for details.

Load the data from CSV files to Bigquery table.

In [5]:
import google.datalab.bigquery as bq

# Create the dataset
bq.Dataset('coast').create()

schema = [
  {'name':'image_url', 'type': 'STRING'},
  {'name':'label', 'type': 'STRING'},
]

# Create the table
train_table = bq.Table('coast.train').create(schema=schema, overwrite=True)
train_table.load('gs://cloud-datalab/sampledata/coast/train.csv', mode='overwrite', source_format='csv')
eval_table = bq.Table('coast.eval').create(schema=schema, overwrite=True)
eval_table.load('gs://cloud-datalab/sampledata/coast/eval.csv', mode='overwrite', source_format='csv')

Job ajhamilton-scratch/job_GrfeXGWMfX-HCG9EP8LUfmXfup3x completed

See the following file for the label description:

In [6]:
!gsutil cat gs://cloud-datalab/sampledata/coast/dict_explanation.csv

In [7]:
%%bq query --name coast_train
SELECT image_url, label FROM coast.train

In [8]:
coast_train.execute().result()

image_url,label
gs://tamucc_coastline/esi_images/IMG_3820_SecDE_Spr12.jpg,1
gs://tamucc_coastline/esi_images/IMG_1074_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_1077_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_1079_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_1084_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_1088_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_0411_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_0324_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_0321_SecABD_Sum12_Pt1.JPG,1
gs://tamucc_coastline/esi_images/IMG_0413_SecABD_Sum12_Pt1.JPG,1


### Explore Your Data

Sample the data to around 1000 instances for visualization. Our data is very simple, so we simply draw histogram on the labels and compare training and eval data.

In [9]:
from google.datalab.ml import *

ds_train = BigQueryDataSet(table='coast.train')
ds_eval = BigQueryDataSet(table='coast.eval')

df_train = ds_train.sample(1000)
df_eval = ds_eval.sample(1000)

TypeError: 'dict_values' object does not support indexing

In [ ]:
df_train.label.value_counts().plot(kind='bar');

In [ ]:
df_eval.label.value_counts().plot(kind='bar');